In [12]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/workspaces/SebastianBot")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from cloud.dependencies.clients import resolve_gmail_client


gmail = resolve_gmail_client()

In [16]:
from datetime import datetime
from sebastian.shared.gmail.query_builder import GmailQueryBuilder


mails = gmail.fetch_mails(GmailQueryBuilder().from_email("info@hautarztpraxis-huhn.de").after_date(datetime(2025, 12, 28)).build())
mail = mails[0]
mail

FullMailResponse(id='19b9257b62f0d752', threadId='19b9257b62f0d752', labelIds=['IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'], snippet='Sehr geehrte Patientin, sehr geehrter Patient, das Rezept wird vorbereitet und kann ab dem nächsten Werktag während unserer Sprechzeiten abgeholt werden. Bitte vergessen Sie ihre Chipkarte der', raw_payload={'partId': '', 'mimeType': 'text/plain', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'oneironaut.oml@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:6a06:6209:20b0:76c:7b96:3dbd with SMTP id ik9-n2csp1154315pib;        Tue, 6 Jan 2026 00:06:20 -0800 (PST)'}, {'name': 'X-Google-Smtp-Source', 'value': 'AGHT+IGyRHqsNKYyTDfp1ASV51cwOAYjOziuyHTWasWE9YAAlHrPsSBN0QB/KutlLJ4+UNdaHvl1'}, {'name': 'X-Received', 'value': 'by 2002:aa7:dac5:0:b0:64b:6271:4e1e with SMTP id 4fb4d7f45d1cf-65079558a67mr1560578a12.17.1767686780544;        Tue, 06 Jan 2026 00:06:20 -0800 (PST)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1767686780; cv=

In [ ]:
html_string =mail.payload
html_string

In [19]:
with open("fetch_recipe.html", "w") as f:
    f.write(html_string)

In [7]:
html_string

''

In [16]:
from bs4 import BeautifulSoup

# Parse the HTML string and extract visible text
soup = BeautifulSoup(html_string, "html.parser")
visible_text = soup.get_text()
visible_text

'DoctolibIhr Termin musste abgesagt werdenDienstag 13 Januar07:30E. LoukaHallo Oliver,leider musste die Gesundheitseinrichtung Ihren Termin absagen.NEUEN TERMIN FINDENKontaktÜber unsHilfebereichDiese E-Mail wird Ihnen durch Doctolib im Auftrag und im Namen Ihrer Gesundheitsfachkraft zugesandt, da Ihre Gesundheitsfachkraft die Dienste von Doctolib nutzt, um ihre Termine zu verwalten. Doctolib handelt als Auftragsverarbeiter und Ihre Gesundheitsfachkraft als verantwortliche Stelle.\r\nUm Ihre Rechte in Bezug auf Ihre persönlichen Daten auszuüben, können Sie sich direkt an Ihre sendende Gesundheitsfachkraft wenden. \r\nUm mehr darüber zu erfahren, wie Doctolib Ihre personenbezogenen Daten verarbeitet, lesen Sie bitte unsere   Datenschutzhinweise.\r\nDiese E-Mail wird durch Doctolib GmbH, Mehringdamm 51, 10961 Berlin, Amtsgericht Charlottenburg HRB 175963B, gesendet.\r\nBitte beachten Sie, dass dies nicht der Ort ist, an dem Ihr Termin stattfindet.\n\n'

In [18]:
from cloud.dependencies.clients import resolve_gemini_client


gemini = resolve_gemini_client()

In [25]:
prompt = f"""You are my personal assistant, managing incoming emails.
Your goal is to decide if an email requires my attention or can be ignored.
You have the following options:
1. Create a task in my calendar. This should be used whenever the mail content means that I need to do something. I.e. attend an appointment, make a call, send an email, or reschedule an appointment.
2. Send a telegram message. This should be used if the mail content is important, but creating a task does not make sense.

You may chose to any number and any combination of the above options, or none at all.
Reply in german.

---Mail content (text only)---
{visible_text}
---End of mail content---"""

In [26]:
from cloud.functions.infrastructure.AllHandler.models import AllHandlerEvent


response = gemini.get_response(prompt, AllHandlerEvent)

In [27]:
response

Result(item=create_task_events=[CreateTaskEvent(title='Neuen Doctolib-Termin finden', notes='Der Termin am Dienstag, 13. Januar um 07:30 Uhr wurde abgesagt.', due=None, task_list_id=<TaskListIds.Default: 'MDY0Mzc2NjgyMDc4MTc0Nzg3Mjk6MDow'>)] send_telegram_message_events=[], errors=[])